In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from itertools import chain
from tensorflow.keras import layers
import matplotlib.pyplot as plt

import matplotlib.image as mpimg


# Suppress TensorFlow warnings for clarity
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)

print("TensorFlow version:", tf.__version__)

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.1),  # Rotates images by a factor between -10% and +10%
    layers.RandomZoom(0.1),
])

labels_df = pd.read_csv('/Users/angel/OneDrive/Desktop/CS 4100/Satellite_Image_Classifier/data/train_v2.csv')
labels_df.head()

# Print all unique tags
# from itertools import chain
labels_list = list(chain.from_iterable([tags.split(" ") for tags in labels_df['tags'].values]))
labels_set = set(labels_list)

images_title = [labels_df[labels_df['tags'].str.contains(label)].iloc[i]['image_name'] + '.jpg' 
                for i, label in enumerate(labels_set)]

# plt.rc('axes', grid=False)
# _, axs = plt.subplots(5, 4, sharex='col', sharey='row', figsize=(15, 20))
# axs = axs.ravel()

# for i, (image_name, label) in enumerate(zip(images_title, labels_set)):
#     img = mpimg.imread('/Users/angel/OneDrive/Desktop/CS 4100/train-jpg/train-jpg' + '/' + image_name)
#     axs[i].imshow(img)
#     axs[i].set_title('{} - {}'.format(image_name, label))


TensorFlow version: 2.17.0


In [35]:
train_jpeg_dir = '/Users/angel/OneDrive/Desktop/CS 4100/train-jpg/train-jpg'
train_csv_file = '/Users/angel/OneDrive/Desktop/CS 4100/Satellite_Image_Classifier/data/train_v2.csv'
test_jpeg_dir = '/Users/angel/OneDrive/Desktop/CS 4100/test-jpg/test-jpg'
test_additional_jpeg_dir = '/Users/angel/OneDrive/Desktop/CS 4100/Satellite_Image_Classifier/data/test_v2_file_mapping.csv'
img_resize = (128, 128)  # Desired image size
validation_split = 0.2
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

# Step 1: Read Labels and File Paths
labels_df = pd.read_csv(train_csv_file)

# Step 2: Extract all unique labels
labels = sorted(set(chain.from_iterable([tags.split(" ") for tags in labels_df['tags'].values])))

# Step 3: Create a mapping from label to index
labels_map = {label: idx for idx, label in enumerate(labels)}
num_classes = len(labels_map)

# Step 4: Map Labels to Integers and One-Hot Encode
def encode_tags(tags_str):
    tags = tags_str.split(' ')
    targets = np.zeros(num_classes, dtype='float32')
    for tag in tags:
        targets[labels_map[tag]] = 1.0
    return targets

labels_df['targets'] = labels_df['tags'].apply(encode_tags)


In [42]:
# Step 5: Split Data into Training and Validation Sets
train_df, val_df = train_test_split(labels_df, test_size=validation_split, random_state=42)

# Reset indices
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

print(len(train_df), len(val_df))

32383 8096


In [7]:
# Step 6: Define the Pure Python Image Loading Function
def _load_image(path_str):
    """
    Load and preprocess an image from a given file path.

    Args:
        path_str (bytes): The file path as a bytes object.

    Returns:
        np.ndarray: The preprocessed image as a NumPy array.
    """
    try:
        from PIL import Image  # Import inside the function to ensure accessibility

        # Decode the bytes to a UTF-8 string
        path_str = path_str.decode('utf-8')

        # Open the image using PIL
        with Image.open(path_str) as img:
            # Convert image to RGB (handles CMYK and other modes)
            img = img.convert('RGB')
            # Resize the image
            img = img.resize(img_resize)
            # Convert to NumPy array and normalize to [0, 1]
            img_array = np.array(img).astype(np.float32) / 255.0
            # Normalize to [-1, 1]
            img_array = (img_array * 2.0) - 1.0

        return img_array
    except Exception as e:
        print(f"Error loading image {path_str}: {e}")
        # Return a zero tensor as a placeholder to prevent pipeline failure
        return np.zeros([img_resize[0], img_resize[1], 3], dtype=np.float32)

In [8]:
# Step 7: Define the TensorFlow Wrapper Function using tf.numpy_function
def load_and_preprocess_image_numpy(path, label):
    """
    TensorFlow wrapper for loading and preprocessing an image using tf.numpy_function.

    Args:
        path (tf.Tensor): The file path as a TensorFlow string tensor.
        label (tf.Tensor): The corresponding label tensor.

    Returns:
        tuple: A tuple of the preprocessed image tensor and label tensor.
    """
    # Use tf.numpy_function to apply the pure Python _load_image function
    image = tf.numpy_function(func=_load_image, inp=[path], Tout=tf.float32)

    # Set the shape of the image tensor
    image.set_shape([img_resize[0], img_resize[1], 3])

    return image, label

In [11]:
def augment_image(image, label):
    """
    Apply data augmentation to the image.
    
    Args:
        image (tf.Tensor): The input image tensor.
        label (tf.Tensor): The corresponding label tensor.
    
    Returns:
        tuple: The augmented image tensor and the original label tensor.
    """
    image = data_augmentation(image)
    return image, label

In [50]:
# Step 8: Define the Dataset Creation Function using tf.numpy_function
def create_dataset_numpy(df, training=True):
    """
    Create a TensorFlow dataset from a DataFrame using tf.numpy_function.

    Args:
        df (pd.DataFrame): DataFrame containing image paths and labels.
        training (bool): Whether the dataset is for training (enables shuffling).

    Returns:
        tf.data.Dataset: The prepared dataset.
    """
    # Construct full file paths
    image_paths = df['image_name'].apply(lambda x: os.path.join(train_jpeg_dir, f"{x}.jpg")).tolist()
    labels = np.stack(df['targets'].values)

    # Create a TensorFlow Dataset from the file paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    # Map the load_and_preprocess_image_numpy function to the dataset
    dataset = dataset.map(load_and_preprocess_image_numpy, num_parallel_calls=AUTOTUNE)

    if training:
        # Shuffle the dataset for training
        dataset = dataset.shuffle(buffer_size=1000)
        # Apply data augmentation
        dataset = dataset.map(augment_image, num_parallel_calls=AUTOTUNE)

    # Batch and prefetch the dataset for optimal performance
    dataset = dataset.batch(batch_size).prefetch(AUTOTUNE)

    return dataset

In [51]:
# Step 9: Create Training and Validation Datasets using tf.numpy_function
train_dataset_numpy = create_dataset_numpy(train_df, training=True)
val_dataset_numpy = create_dataset_numpy(val_df, training=False)

#print(len(train_dataset_numpy),len (val_dataset_numpy))

# Step 10: Verify the Datasets
print("\nVerifying Training Dataset with tf.numpy_function:")
for images, labels in train_dataset_numpy.take(1):
    print("Image tensor shape:", images.shape)
    print("Image tensor dtype:", images.dtype)
    print("Image tensor min value:", tf.reduce_min(images).numpy())
    print("Image tensor max value:", tf.reduce_max(images).numpy())
    print("Image tensor mean value:", tf.reduce_mean(images).numpy())
    break


Verifying Training Dataset with tf.numpy_function:
Image tensor shape: (32, 128, 128, 3)
Image tensor dtype: <dtype: 'float32'>
Image tensor min value: -0.9707219
Image tensor max value: 0.8028034
Image tensor mean value: -0.37111065


In [ ]:
# Step 12: Create Test Dataset using tf.numpy_function and PIL-based Loader
def load_and_preprocess_test_image_numpy(path):
    """
    TensorFlow wrapper for loading and preprocessing test images using tf.numpy_function.

    Args:
        path (tf.Tensor): The file path as a TensorFlow string tensor.

    Returns:
        tf.Tensor: The preprocessed image tensor.
    """
    # Use tf.numpy_function to apply the pure Python _load_image function
    image = tf.numpy_function(func=_load_image, inp=[path], Tout=tf.float32)

    # Set the shape of the image tensor
    image.set_shape([img_resize[0], img_resize[1], 3])

    return image

# Get test file paths
test_files = [os.path.join(test_jpeg_dir, f) for f in os.listdir(test_jpeg_dir)]

print(len(test_files))

# Create test dataset using the PIL-based loader
test_dataset = tf.data.Dataset.from_tensor_slices(test_files)
test_dataset = test_dataset.map(load_and_preprocess_test_image_numpy, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(AUTOTUNE)

# (Optional) Visualize Test Images Without Labels
num_test_images_to_display = 9
plt.figure(figsize=(10, 10))

for images in test_dataset.take(1):
    test_images_np = images.numpy()

    # Convert images from [-1, 1] to [0, 1] for visualization
    test_images_np = (test_images_np + 1.0) / 2.0
    break

print(len(test_images_np))

# for i in range(num_test_images_to_display):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(test_images_np[i])
#     plt.axis("off")
# plt.tight_layout()
# plt.show()

40669
32


<Figure size 1000x1000 with 0 Axes>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models

# Convert dataset into numpy arrays
x_train_data = []
y_train_data = []

x_test_data = []
y_test_data = []

for x_batch, y_batch in train_dataset_numpy:
    x_train_data.append(x_batch.numpy())  # Add images to list
    y_train_data.append(y_batch.numpy())  # Add labels to list

for x_batch, y_batch in val_dataset_numpy:
    x_test_data.append(x_batch.numpy())  # Add images to list
    y_test_data.append(y_batch.numpy())  # Add labels to list

# Concatenate to get all data
x_train_data = np.concatenate(x_train_data, axis=0)
y_train_data = np.concatenate(y_train_data, axis=0)
x_test_data = np.concatenate(x_test_data, axis=0)
y_test_data = np.concatenate(y_test_data, axis=0)

print("All images (x):", x_test_data.shape)
print("All labels (y):", y_test_data.shape)




All images (x): (8096, 128, 128, 3)
All labels (y): (8096, 17)
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]


In [62]:
print(y_test_data[0])
# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_resize[0], img_resize[1], 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='sigmoid')  # Use the number of unique classes
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train_data, y_train_data, epochs=6, batch_size=batch_size, validation_split=0.2)


#predictions = model.predict(x_test_data)
#predicted_classes = np.argmax(predictions, axis=1)

# accuracy = accuracy_score(predicted_classes, y_train_data)

# print(f"Test accuracy: {accuracy}")
from sklearn.metrics import classification_report

# Predict on test data
y_pred_prob = model.predict(x_test_data)

thresholds = [0.5, 0.2, 0.3, 0.2, 0.2, .5, .4, .2, .4, .4, .4,  .5, .5, .5, .2, .2, .5]  # Custom threshold for each class
predicted_classes = np.array([pred > threshold for pred, threshold in zip(y_pred_prob.T, thresholds)]).T.astype(int)

y_pred = (y_pred_prob > 0.5).astype(int)  # Threshold probabilities at 0.5

# Flatten the test labels for comparison
print(classification_report(y_test_data, y_pred, target_names=labels_map))

print(classification_report(y_test_data, predicted_classes, target_names=labels_map))

# Example of printing a prediction
# for i in range(5):  # Print first 5 predictions
#     print(f"Predicted class for test image {i}: {predicted_classes[i]} (Actual class: {y_train_data[i]})")

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]


c:\Users\angel\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/6
810/810 ━━━━━━━━━━━━━━━━━━━━ 67s 82ms/step - accuracy: 0.0487 - loss: 0.2157 - val_accuracy: 0.0594 - val_loss: 0.1662
Epoch 2/6
810/810 ━━━━━━━━━━━━━━━━━━━━ 71s 88ms/step - accuracy: 0.0806 - loss: 0.1541 - val_accuracy: 0.0662 - val_loss: 0.1494
Epoch 3/6
810/810 ━━━━━━━━━━━━━━━━━━━━ 71s 88ms/step - accuracy: 0.0838 - loss: 0.1370 - val_accuracy: 0.0933 - val_loss: 0.1435
Epoch 4/6
810/810 ━━━━━━━━━━━━━━━━━━━━ 71s 88ms/step - accuracy: 0.0897 - loss: 0.1213 - val_accuracy: 0.0721 - val_loss: 0.1407
Epoch 5/6
810/810 ━━━━━━━━━━━━━━━━━━━━ 73s 90ms/step - accuracy: 0.0934 - loss: 0.1063 - val_accuracy: 0.0914 - val_loss: 0.1467
Epoch 6/6
810/810 ━━━━━━━━━━━━━━━━━━━━ 90s 112ms/step - accuracy: 0.0930 - loss: 0.0899 - val_accuracy: 0.0919 - val_loss: 0.1603
253/253 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step
                   precision    recall  f1-score   support

      agriculture       0.77      0.74      0.76      2441
   artisinal_mine       0.49      0.33      0.39        70
     

c:\Users\angel\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\angel\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\angel\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo